In this tutorial, we will take a brief look at the internals of ```CrypTensors```. 

This tutorial is optional, and can be skipped without any loss of continuity to the following tutorials.

In [1]:
#import the libraries
import crypten
import torch

INFO:root:==================
INFO:root:DistributedCommunicator with rank 0
INFO:root:==================
INFO:root:World size = 1


Currently, a ```CrypTensor``` is always a tensor encrypted using secure MPC protocols, called ```MPCTensor```. In order to support the mathematical operations required by the ```MPCTensor```, CrypTen implements two kinds of secret-sharing protocols: arithmetic (or additive) secret-sharing and binary (or XOR) secret-sharing. Arithmetic secret sharing forms the basis of most of the mathematical operations implemented by ```MPCTensor```. On the other hand, binary secret-sharing allows greater efficiency in evaluating logical expressions. 

```MPCTensors``` encrypted with arithmetic secret-sharing are internally ```ArithmeticSharedTensors```. ```MPCTensors``` encrypted with binary secret-sharing are internally ```BinarySharedTensors```. 

In this tutorial, we'll first introduce the concept of a ```CrypTensor``` ```ptype``` (for <i>private-type</i>), and show how to use it to obtain ```ArithmeticSharedTensors``` and ```BinarySharedTensors```. We'll then look more closely at operations supported by ```ArithmeticSharedTensor``` and ```BinarySharedTensor```. Finally, we'll describe how CrypTen handles an operation that needs to use both types of secret-sharing for its tensors.


### <i>ptype</i> in CrypTen
CrypTen defines the ```ptype``` (for <i>private-type</i>) attribute of a ```CrypTensor``` to denote the kind of secret-sharing protocol used in the ```CrypTensor```. The ```ptype``` is, in many ways, analogous to the ```dtype``` of PyTorch. The ```ptype``` may have two values: 
<ul>
<li>```crypten.arithmetic``` for ```ArithmeticSharedTensors```</li>
<li>```crypten.binary``` for  ```BinarySharedTensors```</li>
</ul>
We can use the ```ptype``` attribute to create a ```CrypTensor``` with the appropriate secret-sharing protocol. For example: 

In [2]:
#Constructing CrypTensors with ptype attribute

#arithmetic secret-shared tensors
x_enc = crypten.cryptensor([1.0, 2.0, 3.0], ptype=crypten.arithmetic)
print("x_enc type:", type(x_enc))
print("x_enc internal type:", type(x_enc._tensor))
print()

#binary secret-shared tensors
#TODO: this will change to be analogous to arithmetic secret-shared tensors after removal of assertion
y = torch.tensor([1, 2, 1], dtype=torch.int32)
y_enc = crypten.cryptensor(y, ptype=crypten.binary)
print("y_enc object type:", type(y_enc))
print("y_enc internal type:", type(y_enc._tensor))

x_enc type: <class 'crypten.mpc.mpc.MPCTensor'>
x_enc internal type: <class 'crypten.mpc.primitives.arithmetic.ArithmeticSharedTensor'>

y_enc object type: <class 'crypten.mpc.mpc.MPCTensor'>
y_enc internal type: <class 'crypten.mpc.primitives.binary.BinarySharedTensor'>


### ArithmeticSharedTensor
Let's now look more closely at the ```ArithmeticSharedTensor```. The ```ArithmeticSharedTensor``` implements arithmetic secret sharing, which forms the basis of most of the mathematical operations implemented by ```MPCTensor```. The creation of an ```ArithmeticSharedTensor``` and its use in arithmetic operations is similar to what we've seen in Tutorial 1.

In [3]:
from crypten.mpc.primitives import ArithmeticSharedTensor 

#creating an ArithmeticSharedTensor
x_enc = ArithmeticSharedTensor([1, 2, 3])

#operations with plaintext tensors
y_enc = x_enc + 2
print("addition:", y_enc.get_plain_text())

y_enc = x_enc - 2
print("subtraction:", y_enc.get_plain_text())

y_enc = x_enc * 2
print("multiplication:", y_enc.get_plain_text())

y_enc = x_enc / 2
print("division: ", y_enc.get_plain_text())

addition: tensor([3., 4., 5.])
subtraction: tensor([-1.,  0.,  1.])
multiplication: tensor([2., 4., 6.])
division:  tensor([0.5000, 1.0000, 1.5000])


In [4]:
#operations with encrypted tensors
y_enc = ArithmeticSharedTensor([2])

z_enc = x_enc + y_enc
print("addition:", z_enc.get_plain_text())

z_enc = x_enc - y_enc
print("subtraction:", z_enc.get_plain_text())

z_enc = x_enc * y_enc
print("multiplication:", z_enc.get_plain_text())

z_enc = x_enc / y_enc
print("division:", z_enc.get_plain_text())

addition: tensor([3., 4., 5.])
subtraction: tensor([-1.,  0.,  1.])
multiplication: tensor([2., 4., 6.])
division: tensor([0.5000, 1.0000, 1.5000])


How do we know the ```ArithmeticSharedTensor``` does not directly contain our plaintext tensor? Let's look at the tensor held by an ```ArithmeticSharedTensor```.

In [5]:
x = torch.tensor([4, 5, 6])
print("Plaintext tensor:", x)

#encrypting the plaintext tensor x
x_enc = ArithmeticSharedTensor(x)

#print tensor inside the ArithmeticSharedTensor x_enc
print("Encrypted tensor:", x_enc._tensor)

Plaintext tensor: tensor([4, 5, 6])
Encrypted tensor: tensor([262144, 327680, 393216])


We see that the tensor held by ```x_enc``` is not the plaintext tensor ```x```!

This concludes our overview of ```ArithmeticSharedTensors```. Next, we'll look at ```BinarySharedTensors```.
<br>
### BinarySharedTensors
The second type of secret-sharing implemented in CrypTen is binary or XOR secret-sharing. This type of secret-sharing allows greater efficiency in evaluating logical expressions. 

We can create a ```BinarySharedTensor``` just as we did an ```ArithmeticSharedTensor```. However, the ```BinarySharedTensor``` supports binary logical operations, rather than arithmetic operations. Thus, for example: 

In [6]:
from crypten.mpc.primitives import BinarySharedTensor

#creating a BinarySharedTensor. TODO: fix after assertion removed.
x = torch.tensor([1, 2, 1], dtype=torch.int32)
x_enc = BinarySharedTensor(x)

#operations between BinarySharedTensor/plaintext tensors
y_enc = x_enc ^ 2
print("XOR operation:", y_enc.get_plain_text())

y_enc = x_enc & 2
print("AND operation:", y_enc.get_plain_text())

y_enc = x_enc | 2
print("OR operation:", y_enc.get_plain_text())

XOR operation: tensor([3, 0, 3])
AND operation: tensor([0, 2, 0])
OR operation: tensor([3, 2, 3])


In [7]:
#creating a second BinarySharedTensor
y = torch.tensor([2, 1, 2], dtype=torch.int32)
y_enc = BinarySharedTensor(y)

#operations between two BinarySharedTensors
z_enc = x_enc ^ y_enc
print("XOR operation:", z_enc.get_plain_text())

z_enc = x_enc & y_enc
print("AND operation:", z_enc.get_plain_text())

z_enc = x_enc | y_enc
print("OR operation:", z_enc.get_plain_text())

XOR operation: tensor([3, 3, 3])
AND operation: tensor([0, 0, 0])
OR operation: tensor([3, 3, 3])


As we did with the ```ArithmeticSharedTensor```, we can look into the ```BinarySharedTensor``` to see that it does not actually keep the plaintext tensor. <b>TODO: This does not work with world size=1.</b>

In [8]:
x = torch.tensor([1, 2, 3], dtype=torch.int32)
print("Plaintext tensor:", x)

#Encrypt as BinarySharedTensor
x_enc = BinarySharedTensor(x)

#access the tensor inside the BinarySharedTensor x_enc
print("Encrypted tensor:", x_enc._tensor)

Plaintext tensor: tensor([1, 2, 3], dtype=torch.int32)
Encrypted tensor: tensor([1, 2, 3])


### Using Both Secret-sharing Protocols
Quite often a mathematical function may need to use both additive and XOR secret sharing for efficient evaluation. CrypTen provides functionality that allows for the conversion of ```ArithmeticSharedTensor``` to ```BinarySharedTensor``` and vice-versa. 

In [9]:
from crypten.mpc import MPCTensor

x = torch.tensor([1, 2, 3])
print("Plaintext tensor:", x)
print()

#creating an MPCTensor that uses arithmetic secret sharing
x_enc_arithmetic = MPCTensor(x, ptype=crypten.arithmetic)
print("Initial encrypted type:", type(x_enc_arithmetic._tensor))
print("ArithmeticSharedTensor value:", x_enc_arithmetic._tensor._tensor)
print()

#converting the MPCTensor to use binary secret sharing
#TODO: This does not work with world size=1
x_enc_binary = x_enc_arithmetic.binary()
print("After conversion:", type(x_enc_binary._tensor))
print("BinarySharedTensor value:", x_enc_binary._tensor._tensor)
print()

#Decrypting the converted tensor yields the original tensor
print("Decrypting converted tensor:", x_enc_binary.get_plain_text())

Plaintext tensor: tensor([1, 2, 3])

Initial encrypted type: <class 'crypten.mpc.primitives.arithmetic.ArithmeticSharedTensor'>
ArithmeticSharedTensor value: tensor([ 65536, 131072, 196608])

After conversion: <class 'crypten.mpc.primitives.binary.BinarySharedTensor'>
BinarySharedTensor value: tensor([ 65536, 131072, 196608])

Decrypting converted tensor: tensor([1., 2., 3.])


Thus, even with conversions between ```ArithmeticSharedTensors``` and ```BinarySharedTensors```, CrypTen correctly decrypts the tensor.

For the algorithms that convert between ```ArithmeticSharedTensors``` and ```BinarySharedTensors```, as well as examples of mathematical functions that require such conversion, please see the CrypTen whitepaper.